In [1]:
import networkx as nx
import numpy as np
import random
import osmnx as ox
import folium
from folium.plugins import MarkerCluster
import webbrowser
import requests
from itertools import permutations

In [2]:
API_KEY = 'x'

In [20]:
z = 5
ville = "Paris"
pays = "France"
lieux = ville +', '+ pays

In [21]:
import requests

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}
url = f'https://api.openrouteservice.org/geocode/search?api_key={API_KEY}&text={lieux}'
call = requests.get(url, headers=headers)

if call.status_code == 200:
    data = call.json()
    bbox = data['features'][0]['bbox']  # Accéder à la bbox
    print("Bbox de la ville Avion en France:", bbox)
else:
    print("Erreur lors de la requête:", call.status_code, call.reason)

Bbox de la ville Avion en France: [2.224225, 48.815607, 2.469769, 48.902008]


In [22]:
import requests

body = {"locations":[[2.846550,50.3996],[2.8335,50.3981],[2.842,50.40543]]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62481a74eda827a04828adf4f255bf046343',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"durations":[[0.0,185.38,223.74],[185.38,0.0,291.48],[217.47,291.48,0.0]],"destinations":[{"location":[2.846535,50.399548],"snapped_distance":5.93},{"location":[2.833491,50.39754],"snapped_distance":62.23},{"location":[2.841761,50.405658],"snapped_distance":30.47}],"sources":[{"location":[2.846535,50.399548],"snapped_distance":5.93},{"location":[2.833491,50.39754],"snapped_distance":62.23},{"location":[2.841761,50.405658],"snapped_distance":30.47}],"metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"matrix","timestamp":1707102877190,"query":{"locations":[[2.84655,50.3996],[2.8335,50.3981],[2.842,50.40543]],"profile":"driving-car","responseType":"json"},"engine":{"version":"7.1.1","build_date":"2024-01-29T14:41:12Z","graph_date":"2024-02-04T14:27:02Z"}}}


In [27]:
def get_elevation(coordinates):
    base_url = "https://api.open-elevation.com/api/v1/lookup?locations="

    # Définir la taille maximale de chaque sous-liste
    chunk_size = 50  # Vous pouvez ajuster cette valeur en fonction de vos besoins

    # Diviser la liste en sous-listes de taille chunk_size
    chunks = [coordinates[i:i + chunk_size] for i in range(0, len(coordinates), chunk_size)]

    elevations = []

    # Parcourir chaque sous-liste et effectuer la requête
    for chunk in chunks:
        locations = '|'.join([f"{lat},{lon}" for lat, lon in chunk])
        url = f"{base_url}{locations}"

        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                chunk_elevations = [result['elevation'] for result in data['results']]
                elevations.extend(chunk_elevations)
            else:
                return f"Error: {response.status_code}"
        except Exception as e:
            return f"An error occurred: {e}"

    # Renvoyer simplement la liste des élévations
    return elevations

In [55]:
import json

# Convertir la réponse JSON en dictionnaire Python
response_data = json.loads(call.text)

# Extraire les durées entre les points
durations = response_data['durations']

# Obtenez les élévations pour chaque point (vous pouvez utiliser votre fonction get_elevation)
# Par exemple, en supposant que vous ayez une fonction get_elevation qui prend une liste de coordonnées et renvoie une liste d'élévations :
elevations = get_elevation(body["locations"])

# Définir la vitesse moyenne (en km/h), la consommation de carburant (en kWh/km) et la température (en degrés Celsius)
vitesse_moyenne = 45  # en km/h
consommation_base = 0.2  # en kWh/km (exemple)
temperature = 10  # en degrés Celsius

# Calculer la consommation pour chaque trajet
consommations = []
for i in range(len(durations)):
    for j in range(len(durations[i])):
        temps_trajet = durations[i][j] / 3600  # Convertir la durée de secondes en heures
        distance = vitesse_moyenne * temps_trajet  # Calculer la distance parcourue

        # Calculer la différence d'altitude entre les points de départ et d'arrivée
        difference_altitude = elevations[j] - elevations[i]

        # Ajuster la consommation en fonction de la différence d'altitude et de la température
        ajustement_temperature = min(max((temperature - 15) / 10, -1), 1)  # Ajustement graduel de -20% à +50% pour des températures entre -10°C et 35°C
        consommation = consommation_base * distance * (1 + difference_altitude / 100) * (1 + ajustement_temperature)  # Supposant que chaque 100 mètres de différence d'altitude affecte la consommation de 1%
        consommations.append((i, j, consommation))  # Ajouter les données de consommation à la liste

# Afficher les résultats
for trajet, consommation in enumerate(consommations):
    print(f"Trajet {trajet + 1}: Points {consommation[0]} à {consommation[1]} - Consommation: {consommation[2]:.2f} kWh")


Trajet 1: Points 0 à 0 - Consommation: 0.00 kWh
Trajet 2: Points 0 à 1 - Consommation: 0.23 kWh
Trajet 3: Points 0 à 2 - Consommation: 0.28 kWh
Trajet 4: Points 1 à 0 - Consommation: 0.23 kWh
Trajet 5: Points 1 à 1 - Consommation: 0.00 kWh
Trajet 6: Points 1 à 2 - Consommation: 0.36 kWh
Trajet 7: Points 2 à 0 - Consommation: 0.27 kWh
Trajet 8: Points 2 à 1 - Consommation: 0.36 kWh
Trajet 9: Points 2 à 2 - Consommation: 0.00 kWh


In [56]:
import itertools

# Calculer la consommation totale pour chaque permutation des points
consommations_permutations = []
for permutation in itertools.permutations(range(len(body["locations"]))):
    consommation_totale = sum(consommations[i][j] for i, j in zip(permutation, permutation[1:]))
    consommations_permutations.append((permutation, consommation_totale))

# Trouver la permutation avec la consommation totale la plus faible
meilleure_permutation, meilleure_consommation = min(consommations_permutations, key=lambda x: x[1])

# Afficher l'ordre optimal des points
chemin_optimal = [body["locations"][i] for i in meilleure_permutation]
print("Ordre optimal des points pour minimiser la consommation d'énergie:")
for point in chemin_optimal:
    print(point)
print("Consommation totale d'énergie pour l'itinéraire optimal:", meilleure_consommation, "kWh")


Ordre optimal des points pour minimiser la consommation d'énergie:
[2.8335, 50.3981]
[2.84655, 50.3996]
[2.842, 50.40543]
Consommation totale d'énergie pour l'itinéraire optimal: 0.0 kWh
